# Nepalweb

> use this to get data form nepalweb

In [13]:
import requests
import json
import pandas as pd
from fake_useragent import UserAgent
import time

In [2]:
ua = UserAgent()
headers = {"User-Agent": ua.random}
url = 'https://newweb.nepalstock.com.np/api/nots/nepse-data/floorsheet?page=0&size=500&sort=contractId,desc'
r = requests.get(url, allow_redirects=True,headers=headers)

In [3]:
data = json.loads(r.content)

In [4]:
floorsheet = data['floorsheets']['content']

In [5]:
df = pd.DataFrame(floorsheet)

In [6]:
start_time = time.time()
i = 0
df = pd.DataFrame()


while True:
    headers = {"User-Agent": ua.random}
    url = f'https://newweb.nepalstock.com.np/api/nots/nepse-data/floorsheet?page={i}&size=500&sort=contractId,desc'
    try:
        r = requests.get(url, allow_redirects=True,headers=headers)
        data = json.loads(r.content)
        floorsheet = data['floorsheets']['content']
        if len(floorsheet)==0:
            break
        df_i = pd.DataFrame(floorsheet)
        df = df.append(df_i)
        i+=1
        time.sleep(0.2)
    except:
        time.sleep(1)
        continue
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 27.05197548866272 seconds ---


In [10]:
df = df.drop(['id', 'contractType'], axis=1)

In [12]:
df.to_csv('data.csv', index=False)

# For saved data

In [89]:
df = pd.read_csv('../django_app/data.csv')

In [90]:
df.head()

,contractId,stockSymbol,buyerMemberId,sellerMemberId,contractQuantity,contractRate,contractAmount,businessDate,tradeBookId,stockId,buyerBrokerName,sellerBrokerName,tradeTime,securityName
0,2021060205000158,SHL,25,58,100,230.0,23000.0,2021-06-02,30661551,147,Sweta Securities Private Limited,Naasa Securities Co. Ltd.,2021-06-02T12:05:38.127605,Soaltee Hotel Limited
1,2021060205000157,SHL,34,58,900,230.0,207000.0,2021-06-02,30661550,147,Vision Securities Pvt. Ltd,Naasa Securities Co. Ltd.,2021-06-02T12:05:38.127178,Soaltee Hotel Limited
2,2021060205000156,SHL,34,58,2300,230.0,529000.0,2021-06-02,30661419,147,Vision Securities Pvt. Ltd,Naasa Securities Co. Ltd.,2021-06-02T12:05:22.05489,Soaltee Hotel Limited
3,2021060205000155,SHL,45,58,700,230.0,161000.0,2021-06-02,30661417,147,Imperial Securities Company Pvt. Ltd.,Naasa Securities Co. Ltd.,2021-06-02T12:05:22.054327,Soaltee Hotel Limited
4,2021060205000154,SHL,42,45,120,232.0,27840.0,2021-06-02,30659778,147,Sani Securities Company Ltd.,Imperial Securities Company Pvt. Ltd.,2021-06-02T12:03:04.744962,Soaltee Hotel Limited


In [91]:
df['tradeTime'] = pd.to_datetime(df['tradeTime']).astype(str)